In [5]:
# import libraries
import os
from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import AnalyzeResult
from azure.ai.documentintelligence.models import AnalyzeDocumentRequest
from dotenv import load_dotenv
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient
import pandas as pd
import numpy as np
import openai
from openai import OpenAI

In [4]:
# Load environment variables
load_dotenv()

# change this
endpoint = os.getenv("AZURE_COGNITIVE_ENDPOINT")
key = os.getenv("AZURE_COGNITIVE_KEY")
gpt4_endpoint = os.getenv("OPENAI_ENDPOINT")
gpt4_key = os.getenv("OPENAI_KEY")
openai_key = os.getenv("OPENAI_SEC_KEY")

# Step 1: Preprocessing Large 10K/10Q documents, get Balance Sheets

Edgar Tool

In [28]:
from edgar import *
# Tell the SEC who you are
set_identity("Jessie Zeng lingruiz@andrew.cmu.edu")

[21:08:01] INFO     Identity of the Edgar REST client set to [Jessie Zeng lingruiz@andrew.cmu.edu]      ]8;id=466369;file:///Users/lingruiz/anaconda3/lib/python3.11/site-packages/edgar/core.py\core.py]8;;\:]8;id=949090;file:///Users/lingruiz/anaconda3/lib/python3.11/site-packages/edgar/core.py#161\161]8;;\

In [29]:
def edgarTool_get_filing(ticker, form_type, num_filings):
    # Get the latest 10-K filing for Apple
    filing = Company(ticker).get_filings(form=form_type).latest(num_filings)
    return filing

In [36]:
company_cik = "NWBI"

In [37]:
# get filing by looking up the ticker and form type
filing = edgarTool_get_filing(company_cik, "10-K", 1)

In [38]:
from edgar.financials import Financials
financials = Financials(filing.xbrl())
financials.get_balance_sheet()
financials.get_income_statement()
financials.get_cash_flow_statement()

/Users/lingruiz/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/arrow/array.py:716: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result[valid] = op(np_array[valid], other)
/Users/lingruiz/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/arrow/array.py:716: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result[valid] = op(np_array[valid], other)
/Users/lingruiz/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/arrow/array.py:716: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result[valid] = op(np_array[valid], other)


                                            NORTHWEST BANCSHARES, INC.                                             
                                       Consolidated Statement of Cash Flows                                        
                                                                                                                   
                                                                                     2023        2022        2021  
 ───────────────────────────────────────────────────────────────────────────────────────────────────────────────── 
  Statement of Cash Flows [Abstract]                                                                               
                                                                                                                   
                                                                                                                   
    Operating activities:                                               

## get 3 sheets, export to different formats(csv, json)

In [25]:
# another way to get balance sheet
# tenk = Company("AAPL").get_filings(form="10-K").latest(1).obj()
financials = Financials(filing.xbrl())
balance_sheet_df = financials.get_balance_sheet().get_dataframe().reset_index()
income_statement_df = financials.get_income_statement().get_dataframe().reset_index()
cash_flow_df = financials.get_cash_flow_statement().get_dataframe().reset_index()

/Users/lingruiz/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/arrow/array.py:716: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result[valid] = op(np_array[valid], other)
/Users/lingruiz/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/arrow/array.py:716: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result[valid] = op(np_array[valid], other)
/Users/lingruiz/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/arrow/array.py:716: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result[valid] = op(np_array[valid], other)


In [26]:
balance_sheet_df.head()

,label,2023,2022
0,Statement of Financial Position [Abstract],,
1,ASSETS,,
2,Current assets:,,
3,Cash and cash equivalents,440459000,267397000
4,"Accounts receivable, net of allowance for cred...",1148259000,834513000


In [27]:
def export_csv(df, company_name, form_type):
    df.to_csv(f'../data/10K/{company_name}_{form_type}.csv', index=False)

In [28]:
export_csv(balance_sheet_df, company_cik, "balance_sheet")
export_csv(income_statement_df, company_cik, "income_statement")
export_csv(cash_flow_df, company_cik, "cash_flow")

In [30]:
def dataframe_to_markdown(df: pd.DataFrame) -> str:
    # Create the header row
    header = "| " + " | ".join(df.columns) + " |\n"
    separator = "| " + " | ".join(["---"] * len(df.columns)) + " |\n"
    
    # Create the data rows
    rows = ""
    for _, row in df.iterrows():
        row_text = "| " + " | ".join(map(str, row)) + " |\n"
        rows += row_text
    
    return header + separator + rows


In [31]:
with open(f'../data/10K/{company_cik}_sheet.txt', 'w') as file:
    file.write("\n balance sheet \n")
    file.write(dataframe_to_markdown(balance_sheet_df))
    file.write("\n income statement \n")
    file.write(dataframe_to_markdown(income_statement_df))
    file.write("\n cash flow statment \n")
    file.write(dataframe_to_markdown(cash_flow_df))
    file.close()

# Step 1.1: azure Document Intelligence

Input: a pdf n

Output: location of identified text, table and figures with corresponding text

In [16]:
# sample document
base_path = os.path.abspath(os.path.join(os.getcwd(), '../data'))

formPath = os.path.join(base_path, "10k/plya_consolidated_balance_sheets.pdf")
print(formPath)

/Users/lingruiz/Documents/capstone/data/10k/plya_consolidated_balance_sheets.pdf


In [20]:
def open_pdf(path):
    with open(path, "rb") as f:
        return f.read()


def get_result(formPath, client):
    # read and analyze the document
    poller = client.begin_analyze_document("prebuilt-layout", open_pdf(formPath))
    result = poller.result()
    return result

In [21]:
# create a client
document_analysis_client = DocumentAnalysisClient(
    endpoint=endpoint, credential=AzureKeyCredential(key)
)

In [ ]:
# run the service, save result
result = get_result(formPath, document_analysis_client)

## Explore result structure

In [42]:
# tables are embedded in result.tables, to query the tables, do the following: 
# result.tables returns a list of tables in the pdf document, 
# access row count with row_count, and column count with column_count


for index, table in enumerate(result.tables):
    print(f"table #{index}: \nrow count:{table.row_count}")
    print(f"column count:{table.column_count} \n")

table #0: 
row count:32
column count:3 

table #1: 
row count:32
column count:4 



## Azure result json convert to df/csv

In [45]:
def convert_azdoc_todf(tables):
    tablesCollected = []
    for table_idx, table in enumerate(tables):
        # Initialize an empty matrix
        matrix = [["" for _ in range(table.column_count)] for _ in range(table.row_count)]
        tablesCollected.append(matrix)
        for cell in table.cells:
            row_index = cell.row_index
            column_index = cell.column_index

            if row_index < table.row_count and column_index < table.column_count:
                matrix[row_index][column_index] = cell.content
    
    return tablesCollected

In [93]:
convert_azdoc_todf(result.tables)

[[['', 'As of December 31,', ''],
  ['', '2023', '2022'],
  ['ASSETS', '', ''],
  ['Cash and cash equivalents', '$ 272,520', '$ 283,945'],
  ['Trade and other receivables, net', '74,762', '62,946'],
  ['Insurance recoverable', '9,821', '34,191'],
  ['Accounts receivable from related parties', '5,861', '8,806'],
  ['Inventories', '19,963', '20,046'],
  ['Prepayments and other assets', '54,294', '44,177'],
  ['Property and equipment, net', '1,415,572', '1,536,567'],
  ['Derivative financial instruments', '2,966', '3,510'],
  ['Goodwill, net\n:unselected:', '60,642', '61,654'],
  ['Other intangible assets\n:unselected:', '4,357', '6,556'],
  ['Deferred tax assets', '12,967', '7,422'],
  ['Total assets', '$ 1,933,725', '$ 2,069,820'],
  ["LIABILITIES AND SHAREHOLDERS' EQUITY", '', ''],
  ['Trade and other payables', '$ 196,432', '$ 231,652'],
  ['Payables to related parties', '10,743', '6,852'],
  ['Income tax payable', '11,592', '990'],
  ['Debt', '1,061,376', '1,065,453'],
  ['Other liab

In [19]:
def convert_azdoc_tostring(tables):
    result = ""
    for table_idx, table in enumerate(tables):
        result += f"Table {table_idx + 1}:\n"
        matrix = [["" for _ in range(table.column_count)] for _ in range(table.row_count)]
        
        for cell in table.cells:
            row_index = cell.row_index
            column_index = cell.column_index

            if row_index < table.row_count and column_index < table.column_count:
                matrix[row_index][column_index] = cell.content
        
        # Convert each row in the matrix to a string
        for row in matrix:
            row_str = "\t".join(row)  # Separate columns by tabs for readability
            result += row_str + "\n"
        
        result += "\n"  # Separate tables by a blank line
        
    return result

def convert_azdoc_to_markdown(tables):
    markdown_tables = ""
    for table_idx, table in enumerate(tables):
        # Initialize an empty matrix for the table
        matrix = [["" for _ in range(table.column_count)] for _ in range(table.row_count)]
        
        # Fill in the matrix with cell content
        for cell in table.cells:
            row_index = cell.row_index
            column_index = cell.column_index
            if row_index < table.row_count and column_index < table.column_count:
                matrix[row_index][column_index] = cell.content
        
        # Convert matrix to Markdown format
        markdown_table = "| " + " | ".join(matrix[0]) + " |\n"  # Header row
        markdown_table += "| " + " | ".join(['---' for _ in matrix[0]]) + " |\n"  # Divider row
        
        for row in matrix[1:]:  # Data rows
            markdown_table += "| " + " | ".join(row) + " |\n"
        
        # Add each table to the result with a separating line between tables
        markdown_tables += f"### Table {table_idx + 1}\n" + markdown_table + "\n\n"
    
    return markdown_tables

In [20]:
table_md = convert_azdoc_to_markdown(result.tables)

NameError: name 'result' is not defined

# Step 2: Extract Necessary Data from dataframe

In [11]:
file

<_io.TextIOWrapper name='extract_data_prompt.txt' mode='r' encoding='UTF-8'>

In [11]:
with open('prompts/extract_data_prompt.txt', 'r') as file:
    extract_system_prompt = file.read()
    file.close()

In [13]:
company_cik = "NWBI"
with open(f'markdown_data/{company_cik}_sheet.txt', 'r') as file:
    financial_data = file.read()
    file.close()

In [14]:
from openai import OpenAI
client = OpenAI(api_key=openai_key)

In [15]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": extract_system_prompt},
        {
            "role": "user",
            "content": financial_data
        }
    ]
)
response = completion.choices[0].message


In [16]:
print(response)

ChatCompletionMessage(content='Based on the provided financial statements, I have extracted the requested financial figures and organized them in a structured JSON format. Here are the results:\n\n```json\n{\n  "2023": {\n    "balance_sheet": {\n      "Total Assets": 14419105000,\n      "Current Assets": 1980458000, // Cash and Cash Equivalents and Marketable Securities\n      "Inventory": 0, // Not mentioned in balance sheet\n      "Total Liabilities": 12867788000,\n      "Current Liabilities": 0, // Not directly provided\n      "Shareholders\' Equity": 1551317000\n    },\n    "income_statement": {\n      "Total Revenue": 587922000, // Total interest income\n      "Operating Expenses": 351554000, // Total noninterest expense\n      "Net Income": 134957000\n    },\n    "cash_flow_statement": {\n      "Net Cash Flow from Operating Activities": 92890000,\n      "Net Cash Flow from Investing Activities": -239680000,\n      "Net Cash Flow from Financing Activities": 129685000\n    }\n  },\

In [17]:
response

ChatCompletionMessage(content='Based on the provided financial statements, I have extracted the requested financial figures and organized them in a structured JSON format. Here are the results:\n\n```json\n{\n  "2023": {\n    "balance_sheet": {\n      "Total Assets": 14419105000,\n      "Current Assets": 1980458000, // Cash and Cash Equivalents and Marketable Securities\n      "Inventory": 0, // Not mentioned in balance sheet\n      "Total Liabilities": 12867788000,\n      "Current Liabilities": 0, // Not directly provided\n      "Shareholders\' Equity": 1551317000\n    },\n    "income_statement": {\n      "Total Revenue": 587922000, // Total interest income\n      "Operating Expenses": 351554000, // Total noninterest expense\n      "Net Income": 134957000\n    },\n    "cash_flow_statement": {\n      "Net Cash Flow from Operating Activities": 92890000,\n      "Net Cash Flow from Investing Activities": -239680000,\n      "Net Cash Flow from Financing Activities": 129685000\n    }\n  },\

In [20]:
with open("financial_data.txt", 'w') as f:
    f.write(response.content)
    f.close()

# Step 3: Credit Score Calculations

In [21]:
with open('ratio_calculation_prompt.txt', 'r') as file:
    ratio_calculation_prompt = file.read()
    file.close()

# Step 4: Credit Score Assessment

### OpenAI GPT-4o

In [33]:
file

'\n balance sheet \n| label | 2023 | 2022 |\n| --- | --- | --- |\n| Statement of Financial Position [Abstract] |  |  |\n| ASSETS |  |  |\n| Current assets: |  |  |\n| Cash and cash equivalents | 440459000 | 267397000 |\n| Accounts receivable, net of allowance for credit losses of $82 and $1,753 at June 30, 2023 and 2022, respectively (including amounts receivable from related parties of $5,473 and $8,398 at June 30, 2023 and 2022, respectively) | 1148259000 | 834513000 |\n| Inventories | 1445564000 | 1545606000 |\n| Prepaid expenses and other current assets (including receivables from related parties of $27,732 and $24,412 at June 30, 2023 and 2022, respectively) | 145144000 | 158799000 |\n| Total current assets | 3179426000 | 2806315000 |\n| Property, plant and equipment, net | 290240000 | 285972000 |\n| Deferred income taxes, net | 162654000 | 69929000 |\n| Other assets | 42409000 | 42861000 |\n| Total assets | 3674729000 | 3205077000 |\n| LIABILITIES AND STOCKHOLDERS’ EQUITY |  |  |

In [ ]:
with open('assessment_prompt.txt', 'r') as file:
    assessment_prompt = file.read()
    file.close()

In [ ]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {
            "role": "user",
            "content": file
        }
    ]
)

print(completion.choices[0].message)

In [ ]:
# from openai import AzureOpenAI
    
# client = AzureOpenAI(
#     api_key=gpt4_key,  
#     api_version="2024-08-01-preview",
#     azure_endpoint = gpt4_endpoint
#     )

In [ ]:
# # Define deployment name for the model
# # if needed, add to session chat history to prior prompt the model
# # example use: 
# # {"role": "user", "content": "Does Azure OpenAI support customer managed keys?"},
# #  # {"role": "assistant", "content": "Yes, customer managed keys are supported by Azure OpenAI."},
# # exceeding rate limit will result in 429 error
# deployment_name = 'gpt-4'

# response = client.chat.completions.create(
#     model="gpt-4",
#     messages=[
#         {"role": "system", "content": SYSTEM_PROMPT},
#         {"role": "user", "content": file}
#     ]
# )

# print(response.choices[0].message.content)


In [ ]:
def read_in_text(filename):
    with open(filename, 'r') as file:
        text = file.read()
        file.close()
    return text

def set_openai_client(openai_key):
    client = OpenAI(openai_key)
    return client



def chat_completion(client, model, system_prompt, file, json_format=False):
    
    if json_format:
        completion = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system_prompt},
                {
                    "role": "user",
                    "content": file
                }
            ],
            response_format={"type": "json_object"}
        )
    else:
        completion = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system_prompt},
                {
                    "role": "user",
                    "content": file
                }
            ]
        )
    response = completion.choices[0].message
    return response

def write_response_to_file(response, system_prompt, filename):
    os.mkdir(system_prompt, exist_ok=True)
    with open(f'{system_prompt}/{filename}.txt', 'w') as f:
        f.write(response.content)
        f.close()
    print(f"Response saved to {filename}")


In [ ]:
load_dotenv()
openai_key = os.getenv("OPENAI_SEC_KEY")
model = "gpt-4o-mini"

# step 1: extract data
extract_system_prompt = read_in_text('prompts/extract_data_prompt.txt')
content_file = read_in_text("markdown_data/NWBI_sheet.txt")

response = chat_completion(openai_key, model, extract_system_prompt, content_file, True)
write_response_to_file(response, "extracted_data/financial_data.txt")



# system_prompt = ['extract', 'calculate', 'assess']
def chat_with_model(system_prompt, content_path, openai_key, model = "gpt-4o-mini"):

    system_prompt = read_in_text('prompts/{system_prompt}_prompt.txt')
    content_file = read_in_text("markdown_data/NWBI_sheet.txt")
    response = chat_completion(openai_key, model, extract_system_prompt, content_file, True)
    write_response_to_file(response, 'extract', "financial_data.txt")



NameError: name 'load_dotenv' is not defined

In [27]:
# step 2: ratio calculation
ratio_calculation_prompt = read_in_text('prompts/ratio_calculation_prompt.txt')
content_file = read_in_text("extracted_data/financial_data.txt")

response = chat_completion(openai_key, model, ratio_calculation_prompt, content_file)
write_response_to_file(response, "ratio_calculations/ratio_calculation.txt")


Response saved to ratio_calculations/ratio_calculation.txt


In [ ]:
# step 3: assessment
assessment_prompt = read_in_text('prompts/assessment_prompt.txt')
content_file = read_in_text("ratio_calculations/ratio_calculation.txt")


response = chat_completion(openai_key, model, assessment_prompt, content_file)
write_response_to_file(response, "assessments/assessment_2.txt")

Response saved to assessments/assessment.txt


In [ ]:
# sample read in from config.yml

config = load_config() # function in utils

# Access OpenAI API key
openai_api_key = config["openai"]["api_key"]

# Access scraper settings
base_url = config["scraper"]["base_url"]
timeout = config["scraper"]["timeout"]

print(f"OpenAI API Key: {openai_api_key}")
print(f"Scraper Base URL: {base_url}, Timeout: {timeout}")
